<a href="https://colab.research.google.com/github/DafneGomezFlores/Inteligencia_Artificial/blob/master/parciales/primer%20parcial/LabT1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTAR LIBRERIAS Importa las librerías necesarias para el análisis de datos, aprendizaje automático y visualización.



In [71]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

LEER LOS DATOS DEL DATASET

In [72]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:
# 1. Cargar dataset desde .txt
file_path = "/content/drive/MyDrive/WinnipegDataset.txt"  # Reemplaza con tu ruta
df = pd.read_csv(file_path, sep=None, engine='python')  # Auto detecta el separador

In [74]:
df.head()

,label,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174
0,1,-13.559,-21.407,-11.4040,-15.248,-11.923,-15.291,-2.1548,-7.8474,-10.0020,...,0.18519,0.72602,5.3333,6.000000,0.29489,9.77780,2.44440,1.67700,0.20988,0.65422
1,1,-12.802,-20.335,-10.3990,-14.132,-11.096,-14.361,-2.4039,-7.5330,-9.9369,...,0.33333,-0.48751,2.1111,0.098765,0.83333,0.33333,0.33333,0.84869,0.50617,-0.18898
2,1,-12.431,-19.902,-10.0740,-13.598,-10.829,-14.048,-2.3566,-7.4717,-9.8283,...,0.25926,0.25298,2.2222,0.172840,0.68889,0.88889,0.66667,1.27300,0.30864,0.10483
3,1,-12.689,-19.529,-10.0280,-13.350,-11.056,-14.014,-2.6611,-6.8396,-9.5006,...,0.16049,0.43750,4.1111,0.320990,0.83333,0.33333,0.33333,1.14910,0.38272,0.41603
4,1,-12.686,-19.278,-9.8185,-13.108,-10.932,-13.939,-2.8675,-6.5919,-9.4594,...,0.18519,0.35000,4.0000,0.444440,0.68889,0.88889,0.66667,1.58110,0.20988,0.50000


In [75]:
#df = pd.read_csv(file_path, sep=None, engine="python")

PREPROCESAMIENTO DE DATOS Prepara los datos para el entrenamiento del modelo, incluyendo la separación de características y etiquetas, escalado de características y división en conjuntos de entrenamiento y prueba.

In [76]:
# 2. Features y labels
X = df.drop("label", axis=1).values
y = df["label"].values - 1  # Si tus labels son 1-5, los bajamos a 0-4

In [77]:
print(y)

[0 0 0 ... 6 6 6]


In [78]:
print(X)

[[-13.559   -21.407   -11.404   ...   1.677     0.20988   0.65422]
 [-12.802   -20.335   -10.399   ...   0.84869   0.50617  -0.18898]
 [-12.431   -19.902   -10.074   ...   1.273     0.30864   0.10483]
 ...
 [ -1.927   -11.416    -2.4354  ...   1.273     0.30864   0.31623]
 [  0.12483 -10.144    -0.62193 ...   0.34883   0.80247   0.18898]
 [  0.20063 -10.05     -0.59892 ...   2.0432    0.1358    0.72732]]


In [79]:
# 3. Escalar features
scaler = StandardScaler() #escalar de un rango especifico entre 0 y 1
X = scaler.fit_transform(X)

In [80]:
# 4. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [81]:
# 5. Dataset personalizado
class CustomDataset(Dataset):
    def __init__(self, X, y): #inicializar atributos
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X) # devuelve la longitud

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx] #elementos del dataset

# Crear instancias del dataset (después de la definición de la clase)
train_ds = CustomDataset(X_train, y_train)
test_ds = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)

DEFINICION DEL MODELO Crea una red neuronal simple (MLPClassifier) con una capa oculta.

In [82]:
# 6. Modelo simple
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(MLPClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        return self.model(x)

input_dim = X.shape[1]
model = MLPClassifier(input_dim=input_dim, hidden_dim=128, num_classes=7)


ENTRENAMIENTO DEL MODELO Entrena la red neuronal utilizando los datos de entrenamiento y optimiza los parámetros del modelo para minimizar la pérdida.

In [83]:
# 7. Entrenamiento
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)  # Mover datos al dispositivo
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


EVALUACION DEL MODELO Evalúa el rendimiento del modelo en los datos de prueba utilizando métricas como el informe de clasificación y la matriz de confusión.

In [29]:
# 8. Evaluación
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        outputs = model(batch_X)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(batch_y.numpy())


NameError: name 'model' is not defined

In [30]:
# 9. Reporte de métricas
print("\nClassification Report:\n")
print(classification_report(all_labels, all_preds, digits=4))


Classification Report:



NameError: name 'all_labels' is not defined

VISUALIZACION DE RESULTADOS Muestra la matriz de confusión para visualizar el rendimiento del modelo.

In [2]:
# 10. Matriz de confusión
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(1, 6), yticklabels=range(1, 6))
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

Mounted at /content/drive
